# What is NLP?

Natural Language Processing: The ability of a computer program to understand human language as it is spoken and written.

Example: OpenAI GPT-3

# What is NLU?
Natural Language Understanding

Difference?

Please crack the car window, it is getting hot.

nlp will literally crack the window.
nlu will understand to slightly open the window.

# Sequence Problems
1. one to one
2. one to many
3. many to one
4. many to many
5. many to many synchronized

Use Cases:
* Classification
* Machine Translation
* Text Generation
* Voice Assistants


## What is an RNN

Recurrent Neural Network (RNN) is a type of neural network that is capable of learning to predict the next element in a sequence.

## Architecture of an RNN

Typical architecture of an RNN is as follows:
1. Input Layer
2. Text vectorization layer
3. Embedding layer
4. RNN Cell: LSTM layer
    * Tanh activation function
5. Hidden Activation layer
6. Output Layer
    * Sigmoid
7. Creation of Model
8. Compile
9. Fit


In [24]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3070 Laptop GPU (UUID: GPU-a72a565b-b891-b764-c0f7-6096f5184f0e)


In [25]:
# Helper Functions
from _helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys, walk_through_dir

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

[Source](https://www.kaggle.com/c/nlp-getting-started)

In [26]:
!wget -nc -P ../Downloads/ https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

File ‘../Downloads/nlp_getting_started.zip’ already there; not retrieving.



In [27]:
# Unzip
unzip_data('../Downloads/nlp_getting_started.zip', '../Downloads/08_NLP')

# Walkthrough dir
walk_through_dir('../Downloads/08_NLP')

There are 0 directories and 3 files in '../Downloads/08_NLP'.


## Visualizing a text dataset

To visualize text samples, we need to read them in.

Python read-write
Pandas

In [28]:
import pandas as pd

train_df = pd.read_csv('../Downloads/08_NLP/train.csv')
test_df = pd.read_csv('../Downloads/08_NLP/test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [29]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [30]:
# Check test dataframe
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [31]:
# How many examples of each class?
train_df.target.value_counts()
# 0 = not disaster, 1 = disaster

0    4342
1    3271
Name: target, dtype: int64

In [32]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [33]:
# Visualize random training examples
import random

random_index = random.randint(0, len(train_df) - 5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target == 1 else "(not disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not disaster)
Text:
@engineshed Great atmosphere at the British Lion gig tonight. Hearing is wrecked. http://t.co/oMNBAtJEAO

---

Target: 1 (real disaster)
Text:
The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/aueZxZA5ak

---

Target: 1 (real disaster)
Text:
24 killed in two simultaneous rail crash as acute floods derail the two trains #India #mumbai... http://t.co/b0ZwI0qPTU

---

Target: 0 (not disaster)
Text:
The 1st time someone blew up my phone 30 times they would be blocked. Believe it.  #Catfish

---

Target: 0 (not disaster)
Text:
How many cars do those cyclists wreck going past scratching with pedals. They should be banned #c4news

---



### Split data into training and validation sets

In [34]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, 
                                                                            random_state=42)

In [35]:
# Check lengths

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [36]:
# Check first 10
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

### Converting text into numbers

Tokenization vs Embedding

In NLP, there are two main concepts for turning text into numbers:
* **Tokenization** - A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:
  1. Using **word-level tokenization** with the sentence "I love TensorFlow" might result in "I" being `0`, "love" being `1` and "TensorFlow" being `2`. In this case, every word in a sequence considered a single **token**.
  2. **Character-level tokenization**, such as converting the letters A-Z to values `1-26`. In this case, every character in a sequence considered a single **token**.
  3. **Sub-word tokenization** is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple **tokens**.
* **Embeddings** - An embedding is a representation of natural language which can be learned. Representation comes in the form of a **feature vector**. For example, the word "dance" could be represented by the 5-dimensional vector `[-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]`. It's important to note here, the size of the feature vector is tuneable. There are two ways to use embeddings: 
  1. **Create your own embedding** - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as [`tf.keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)) and an embedding representation will be learned during model training.
  2. **Reuse a pre-learned embedding** - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.


![](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/08-tokenization-vs-embedding.png)
*Example of **tokenization** (straight mapping from word to number) and **embedding** (richer representation of relationships between tokens).*

It depends on your problem. You could try character-level tokenization/embeddings and word-level tokenization/embeddings and see which perform best. You might even want to try stacking them (e.g. combining the outputs of your embedding layers using [`tf.keras.layers.concatenate`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/concatenate)). 

If you're looking for pre-trained word embeddings, [Word2vec embeddings](http://jalammar.github.io/illustrated-word2vec/), [GloVe embeddings](https://nlp.stanford.edu/projects/glove/) and many of the options available on [TensorFlow Hub](https://tfhub.dev/s?module-type=text-embedding) are great places to start.

In [50]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization


sent_lens = [len(sentence.split()) for sentence in train_sentences] 
 
# Taking the 95% length as max length
max_len = int(np.percentile(sent_lens, 95))

print('max len:', max_len)

max len: 24


In [53]:
# Find the avg number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocab

text_vectorizer = TextVectorization(
                    max_tokens=max_vocab_length, # how many words in the vector
                    standardize="lower_and_strip_punctuation", # standardize text
                    split="whitespace", # split text into words via whitespace
                    ngrams=None, # create groups of n-words
                    output_mode="int", # how to map tokens to numbers
                    output_sequence_length=max_len, # how long the output sequence should be
                    pad_to_max_tokens=True, # pad the output sequence to the max length
                )

2022-08-29 11:58:50.764137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [54]:
# Fit the text vectorizer to the training text       
text_vectorizer.adapt(train_sentences)

In [55]:
# Create a sample sentence and tokenize it

sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 24), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>

In [65]:
# Choose a random setnence from the training dataset and tokenize it
random_index = random.randint(0, len(train_sentences) - 1)
random_sentence = train_sentences[random_index]
vectorized_sentence = text_vectorizer([random_sentence])
print(f'original sentence: {random_sentence}\nVectorized: {vectorized_sentence[0]}')

original sentence: Photo: Bath &amp; Body Works cosmetic bag in periwinkle blue with copper piping along the top and four corners.... http://t.co/A9BNlse6QB
Vectorized: [ 343 4111   35   83 4230    1  322    4 9852  824   14    1 9769 1227
    2  212    7 1209 3970    1    0    0    0    0]


In [69]:
# Get the unique words in the vocab

words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the top 5 words (most common)
bottom_5_words = words_in_vocab[-5:] # get the bottom 5 words (least common)

print(f'Number of words in vocab: {len(words_in_vocab)}\nTop 5 words: {top_5_words}\nBottom 5 words: {bottom_5_words}')

Number of words in vocab: 10000
Top 5 words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']
